# Modelling

#### Imports

In [ ]:
import numpy as np
import pandas as pd
import pickle 
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers

import plotly.graph_objs as go
from datetime import datetime
import nbformat

import plotly
from plotly.subplots import make_subplots

import random
random.seed(2505)

#### Load data

In [ ]:
with open('Data/heatpump/data_heatpump_cleaned_v1.pkl', 'rb') as f:
    load_dict = pickle.load(f)

with open('Data/weather/data_weather_v1.pkl', 'rb') as f:
    weather_data = pickle.load(f)

____

### Merging load and weather data

In [ ]:
building_info = pd.read_excel("Data/Gebaeudeinformationen_cleaned.xlsx", index_col=0)
building_info.set_index("Building number", inplace=True)

# add building information
for house in load_dict:
    id = int(re.findall(r'\d+', house)[0])

    load_dict[house]["area"] = building_info.loc[id]["Building area"]
    load_dict[house]["inhabitants"] = building_info.loc[id]["Number of inhabitants"]
    load_dict[house]["building"] = id
    
    weather_data = weather_data[weather_data.index>=1528965900]
    load_dict[house] = pd.concat([load_dict[house], weather_data], axis=1)
    
# concat consumption and weather data
df = pd.concat(load_dict)
df = df.reset_index().set_index('index').sort_index().drop(columns=["level_0"])
df

In [ ]:
df = df[df["building"]==5]
df

____

### Trainieren des Modells

In [ ]:
import numpy as np # linear algebra
from numpy import newaxis
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import LSTM, GRU
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

Datennormierung

In [ ]:
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Erstellung von Sequenzen
## Variante 1: getrennte Zielvariablen
def create_sequences(data, sequence_length):
    sequences = []
    outputs_p = []
    outputs_pf = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        outputs_p.append(data.iloc[i+sequence_length]['P_TOT'])
        outputs_pf.append(data.iloc[i+sequence_length]['PF_TOT'])

    return np.array(sequences), np.array(outputs_p), np.array(outputs_pf)

#sequence_length = 7  # Beispiel für eine Woche
#X, y_p, y_pf = create_sequences(df_scaled, sequence_length)

## Variante 2: gemeinsame Zielvariable
def create_sequences(data, sequence_length):
    sequences = []
    outputs = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        outputs.append(data.iloc[i+sequence_length][['P_TOT', 'PF_TOT']].values)

    return np.array(sequences), np.array(outputs)

sequence_length = 7
sequence_length = sequence_length * 4 * 24
X, y = create_sequences(df_scaled, sequence_length)

# Überprüfen der Dimensionen
print("Shape von X:", X.shape)
print("Shape von y:", y.shape)

# Visualisieren einiger Sequenzen
for i in range(3):  # Die ersten 3 Sequenzen
    print(f"Sequenz {i}:")
    print(X[i])
    print(f"Zugehörige Zielwerte: {y[i]}")
    print("\n")

# Überprüfen der letzten Sequenz
print("Letzte Sequenz und zugehöriger Zielwert:")
print(X[-1])
print(y[-1])


In [ ]:
X.shape

Modell-Definition

In [ ]:
hidden_layers = False

In [ ]:
model = Sequential()

# Eingabeschicht
model.add(GRU(units=50,                 # Hyperparameter -> kann variiert und angepasst werden
              return_sequences=False,   # Konfigurationsparameter, default: False
                                        # Funktionalität:   bestimmt, ob die Schicht einen Ausgabevektor für jeden Zeitpunkt in der Eingabesequenz (return_sequences=True) 
                                        #                   oder nur für den letzten Zeitpunkt (return_sequences=False) zurückgeben soll
                                        #   False: gibt Ausgabevektor für den letzten Zeitschritt zurück: (Anzahl der Beispiele, Anzahl der Units)
                                        #   True:  gibt Ausgabevektor für jeden Zeitschritt in der Eingabesequenz zurück: (Anzahl der Beispiele, Anzahl der Zeitschritte, Anzahl der Units)
                                        # Anwendung: return_sequences=True: mehrere rekurrente Schichten hintereinander (damit jede Schicht eine Sequenz an die nächste weitergibt),  Ausgabe des Modells ist selbst eine Sequenz; 
                                        #            ansonsten: return_sequences=False.
                                        # 
              input_shape=(
                  X.shape[1],           # Sequenzlänge
                  X.shape[2]            # Anzahl der Features
                )
            )
        )
# Dropout-Schicht
model.add(Dropout(0.2))  # Optional: Dropout zur Vermeidung von Overfitting durch zufälliges Deaktivieren von Neuronen während des Trainingsprozesses -> Vermeidung dominanter Neuronen -> bessere Generalisierung

if hidden_layers:
    # weitere GRU-Schichten
    # optional
    model.add(GRU(50, return_sequences=True))
    model.add(Dropout(0.2))

    # Hinzufügen der letzten GRU-Schicht ohne return_sequences (optional, bei der Verwendung mehrerer GRU-Schichten notwendig)
    model.add(GRU(50))
    model.add(Dropout(0.2))

    # Hinzufügen einer Dense-Schicht zur weiteren Merkmalsextraktion (optional)
    model.add(Dense(50, activation='relu'))

# Ausgabeschicht
model.add(Dense(2))  # Zwei Units für die zwei Zielvariablen

In [ ]:
# Visualisierung der Modellstruktur
model.summary()
# plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

Kompilieren des Models

In [ ]:
# Konfigurieren des Modells für das Training -> Festlegung der Lernart sowie die Bewertung des Trainingsprozesses
model.compile(
    optimizer='adam',           # der Optimizer ist ein Algorithmus zur Aktualisierung des Netzwerks, wobei die Gewichte des Modells so angepasst werden, dass Verluste minimiert werden
                                # Verschiedene Optimierer haben unterschiedliche Eigenschaften:
                                # Adam - adaptive moment estimation: Grundprinzipien
                                #   - Adaptive Lernraten: Lernrate wird für jeden Parameter individuell angepasst, basierend auf der Schätzung des ersten Mittelwert und des zweiten Moments der Gradienten
                                #   - Moment-Schätzungen:   > erstes Moment (Mittelwert): Berechnung expontentiell abnehmender Durchschnittswerte vergangener Gradienten -> Steuerung zu relevanten Richtung des Gradientenabstiegs
                                #                           > zweites Moment(Varianz): Berechnung exponentiuell abnehmender Durchschnittswerte vergangener quadrierter Gradienten 
                                #                                                       -> Adaption der Lernrate, Regulierung der Schrittgröße basierend auf der Unsicherheit des Gradienten
                                #   - Korrektur der Bias: Verhinderung der Tendenz, das Schätzungen zu Beginn gegen 0 gehen
                                # Vorteile: Effizienz, wenigeer manuelle Einstellung der Lernrate, gute Performance bei großen Datenmengen/vielen Parametern
    loss='mean_squared_error',  # Verlustfunktion, misst die Genauigkeit des Modells. MSE misst die durchschnittliche quadratische Abweichung zwischen den vorhergesagten und den tatsächlichen Werten
    metrics=['mean_absolute_error']      # Metriken, die für das Training bewertet werden sollen, weitere Alternativen: 'accuracy', ...
    )

Trainieren des Models

In [ ]:
# Aufteilung in Trainings-, Test- und Validierungsdaten
# Ansatz: Chronologische Aufteilung ohne Überlappung
# sonst: sklearn.train_test_split() mit zufälliger Aufteilung der Daten -> Problem: Vorhersage der Vergangenheit mit Werten aus der Zukunft?

train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.85)

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:val_size], y[train_size:val_size]
X_test, y_test = X[val_size:], y[val_size:]

In [ ]:
from time_series_generator import TimeSeriesGenerator

# Erstellen des Generators
SEQUENCE_LENGTH = 7*4*24
train_generator = TimeSeriesGenerator(X_train, y_train, length=SEQUENCE_LENGTH, batch_size=32)
val_generator = TimeSeriesGenerator(X_val, y_val, length=SEQUENCE_LENGTH, batch_size=32)

In [ ]:
# Trainieren des Modells
history = model.fit(
    X_train, y_train,                  # Übergabe der Trainingsdaten
    #train_generator,
    epochs=50,                          # Anzahl der Durchläufe des gesamten Trainingsdatensatzes
                                        #   -> Einfluss: Mehr Epochen können zu einer besseren Anpassung des Modells führen <-> Gefahr des Overfittings
    batch_size=64,                      # Bestimmt die Anzahl der verwendeten Datenpunkte für eine Iteration, bevor die Modellgewichte aktualisiert werden
                                        #   -> größere Batch-Größen: stabilere, aber langsamer konvergierende Updates <-> kleinere Batch-Größen: schnellere, weniger stabile Updates
    validation_data=(X_val, y_val),     # Validierungsdaten, ermöglichen die Überwachung des Trainingsprozesses -> Erkennung von Overfitting
    #validation_data=val_generator,
    use_multiprocessing=True,           # Laufzeitoptimierung
    workers=6,                          # Nutzen mehrerer CPU-Kerne
    verbose=1,                          # Steuert die Menge an Infos, welche während des Trainings ausgegeben werden -> verbose=1 zeigt den Fortschritt für jede Epoche an
)

Speichern des trainierten Modells

In [ ]:
model.save('models/GRU_1layer_30_nodes.h5')  # Speichert das Modell im HDF5-Format

In [ ]:
from keras.models import load_model
model = load_model('models/GRU_1layer_30_nodes.h5')

In [ ]:
### TODO: Wie speicher ich die Ergebnisse?

Visualisierung der Trainingsverlaufs

In [ ]:
from plot_functions import plot_validation_loss
from test import plot_pred_vs_act_cons

In [ ]:
plot_validation_loss(history, "- Model 1")

Evaluierung des Modells mit den Testdaten

In [ ]:
test_loss = model.evaluate(X_test, y_test)
print('Testverlust:', test_loss)
# Vorhersagen auf den Testdaten machen
predicted_test = model.predict(X_test)
print("Shape predicted_test {}".format(predicted_test.shape))
print("Shape y_test {}".format(y_test.shape))

In [ ]:
# Rückskalierung der Daten
# Da der scaler auf alle Trainingsdaten angewendet worden ist, müssen die in den Vorhersagedaten fehlenden Spalten (Wetterdaten und Gebäudedaten) aufgefüllt werden
predicted_test_inversed = scaler.inverse_transform(
    np.hstack((predicted_test, np.zeros((predicted_test.shape[0], df.shape[1] - 2))))
)[:, :2]

y_test_inversed = scaler.inverse_transform(
    np.hstack((y_test, np.zeros((y_test.shape[0], df.shape[1] - 2))))
)[:, :2]

# Zeitstempel
timestamps = df.index
test_timestamps = timestamps[val_size + sequence_length:]

In [ ]:
plot_pred_vs_act_cons(y_test_inversed, predicted_test_inversed, test_timestamps, "2020-12-30")

In [ ]:
from test import print_metrics
print_metrics(y_test_inversed, predicted_test_inversed)